In [1]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy.sql import select, insert, update, delete, join, func
import sqlalchemy

In [ ]:
""" 
<HOMEWORK>
1. POST에 본문 insert
2. insert된 PK 가져오기
3. 위에 코드 이용해서 해시태그 PK값 가져오기
4. POSTAG에 POST.PK, HASHTAG.PK를 insert
5. HASHTAG count = HASHTAG count + 1, if PK == HASHTAG.PK 
 """

# engine, connect Define
engine = create_engine('sqlite:///:memory:', echo = True)
meta = MetaData()
con = engine.connect()

# 초기화
meta.drop_all(engine)
meta.clear()
#if A:
#    meta.remove(A)
#if B:
#    meta.remove(B)
#if C:
#    meta.remove(C)

# 테이블 생성
Table('POST', meta,
      Column('PK', Integer, primary_key= True),
      Column('CONTENT', Text),
      extend_existing = True)

Table('HASHTAG', meta,
      Column('PK', Integer, primary_key= True),
      Column('NAME', Text, nullable=False),
      # server_default는 서버에 디폴트를 저장해서 새로 만들때마다 적용한다.
      Column('Count', Integer, server_default='0'),
      extend_existing = True)

Table('POSTTAG', meta,
      Column('PFK', None, ForeignKey('POST.PK')),
      Column('HFK', Integer, ForeignKey('HASHTAG.PK')),
      extend_existing = True)

# commit
meta.create_all(engine)

# Define Table
A = meta.tables['POST']
B = meta.tables['HASHTAG']
C = meta.tables['POSTTAG']

# 1. POST에 content 넣기 +  2. PK값 불러오기
data1 = {'content': '본문1', 'tags':['Python']}

# A에 컨탠츠를 넣음과 동시에 lastrowid로 PFK 값 추출
A_PK = con.execute(A.insert().values(CONTENT = data1['content'])).lastrowid
print(A_PK)

# 3. 해시태그 PK 값 매칭
for tag in data1['tags']:
    size = con.execute(select(func.count(B.c.Count)).where(B.c.NAME == tag)).fetchone()[0]
    if size == 0:
        con.execute(insert(B).values(NAME = tag))

H_PK = con.execute(select(B.c.PK).where(B.c.NAME == tag)).fetchall()[0]

# 4. POSTAG에 POST.PK, HASHTAG.PK를 insert
con.execute(insert(C).values([{'PFK':A_PK, 'HFK':H_PK[0]}]))

# 5. HASHTAG count = HASHTAG count + 1, if PK == HASHTAG.PK 
con.execute(update(B).where(B.c.PK == C.c.HFK).values({'Count': B.c.Count + 1}))

con.execute(select(B)).fetchall()

In [2]:
# 메타데이터를 넣어주는 객체를 만들어줌
from sqlalchemy.orm import declarative_base

In [3]:
base = declarative_base()

In [4]:
base.metadata.tables # core Table 객체들 - RDMBS와 싱크가 맞음.

FacadeDict({})

In [ ]:
base.registry # object 등록

In [5]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy.sql import select, insert, update, delete, join, func
import sqlalchemy

In [ ]:
# 만약 base 잘못 넣었을 경우, 아래 코드로 날려주고 다시 만든다.
# 객체 제거 + 객체가 저장된 테이블 삭제
base.metadata.remove(base.metadata.tables['USER'])
base.registry.dispose()

In [6]:
class User(base):    
    # object에서 접근할 수 있는 테이블 이름
    # 실제 DBMS에서 사용될 Table 이름
    __tablename__ = 'USER'
    # object에서 pk
    # 실제 DBMS에서 Table에 사용되는 Column이 됨
    pk = Column('PK', Integer, primary_key= True)
    name = Column('NAME', Text)

    def __repr__(self):
        return f'PK: {self.pk}, NAME: {self.name}'

In [8]:
base.metadata.tables

FacadeDict({'USER': Table('USER', MetaData(), Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False), Column('NAME', Text(), table=<USER>), schema=None)})

In [9]:
# 객체 만들기
user1 = User(name = '아무거나')

In [11]:
# main에 등록된 클래스 > 테이블의 이름은 아니다.
type(user1), type(base.metadata.tables['USER'])

(__main__.User, sqlalchemy.sql.schema.Table)

In [13]:
engine = create_engine('sqlite:///:memory:', echo = True)

In [14]:
# create all과 같게 DB에 넣는 작업
base.metadata.create_all(engine)

2023-09-07 09:24:39,405 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:24:39,406 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-07 09:24:39,406 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:24:39,407 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("USER")
2023-09-07 09:24:39,409 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:24:39,412 INFO sqlalchemy.engine.Engine 
CREATE TABLE "USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	PRIMARY KEY ("PK")
)


2023-09-07 09:24:39,413 INFO sqlalchemy.engine.Engine [no key 0.00074s] ()
2023-09-07 09:24:39,414 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
# session이 engine.connect.execute를 DBMS로 알아서 보낸다.
from sqlalchemy.orm import sessionmaker

In [16]:
# in-memory에 있는 object들과 RDBMS 사이에 창구 역할을 한다.
Session = sessionmaker(engine)
sess = Session()

In [17]:
# session을 통해 객체를 등록한다.
sess.add(user1)

In [18]:
user1

PK: None, NAME: 아무거나

In [19]:
# commit을 통해 sess이 DBMS에 반영한다.
# session이라는 얘가 관찰하고 있는 object를 DBMS에 반영한다.
sess.commit()

2023-09-07 09:29:31,299 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:29:31,301 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?)
2023-09-07 09:29:31,302 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ('아무거나',)
2023-09-07 09:29:31,304 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
user1

2023-09-07 09:29:34,485 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:29:34,487 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-07 09:29:34,488 INFO sqlalchemy.engine.Engine [generated in 0.00097s] (1,)


PK: 1, NAME: 아무거나

In [22]:
# 수정확인
sess.is_modified(user1)

False

In [23]:
sess.dirty

IdentitySet([])

In [24]:
user1.name = '다른값'

In [25]:
user1

PK: 1, NAME: 다른값

In [26]:
# 작업할 것을 쌓아줌
sess.dirty

IdentitySet([PK: 1, NAME: 다른값])

In [27]:
# 수정이 필요한지 확인
sess.is_modified(user1)

True

In [28]:
# 반영
sess.commit()

2023-09-07 09:33:02,798 INFO sqlalchemy.engine.Engine UPDATE "USER" SET "NAME"=? WHERE "USER"."PK" = ?
2023-09-07 09:33:02,799 INFO sqlalchemy.engine.Engine [generated in 0.00147s] ('다른값', 1)
2023-09-07 09:33:02,800 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
# name이 바뀐 것을 확인
user1

2023-09-07 09:33:06,956 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:33:06,957 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-07 09:33:06,958 INFO sqlalchemy.engine.Engine [cached since 212.5s ago] (1,)


PK: 1, NAME: 다른값

In [30]:
# 어제 executemany 처럼 []로 묶어서 보낸 작업을 session에서도 동일하게 한다.
sess.add_all([User(name = '2'), User(name = '3')])

In [31]:
sess.commit()

2023-09-07 09:34:26,216 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 09:34:26,217 INFO sqlalchemy.engine.Engine [generated in 0.00010s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('2',)
2023-09-07 09:34:26,226 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 09:34:26,228 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('3',)
2023-09-07 09:34:26,230 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
userList = sess.query(User).all()

2023-09-07 09:34:49,362 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:34:49,364 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER"
2023-09-07 09:34:49,365 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ()


In [33]:
len(userList)

3

In [34]:
user1 is userList[0]

True

In [35]:
userList[1]

PK: 2, NAME: 2

In [36]:
userList[1].name = '다른값2'

In [37]:
# dirty에 무언가 있다는 것은 DB와 싱크가 안맞는 것 > commit 필요
sess.dirty

IdentitySet([PK: 2, NAME: 다른값2])

In [38]:
sess.commit()

2023-09-07 09:35:43,514 INFO sqlalchemy.engine.Engine UPDATE "USER" SET "NAME"=? WHERE "USER"."PK" = ?
2023-09-07 09:35:43,515 INFO sqlalchemy.engine.Engine [cached since 160.7s ago] ('다른값2', 2)
2023-09-07 09:35:43,516 INFO sqlalchemy.engine.Engine COMMIT


In [39]:
userList[1]

2023-09-07 09:35:54,537 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:35:54,538 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-07 09:35:54,539 INFO sqlalchemy.engine.Engine [cached since 380.1s ago] (2,)


PK: 2, NAME: 다른값2

In [43]:
# Python에서 작업하여 값 확인
[user for user in userList if user.name == '3'][0] is userList[2]

True

In [44]:
# DB에서 작업하여 값 확인
sess.query(User).where(User.name == '3').one() is userList[2]

2023-09-07 09:39:09,457 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."NAME" = ?
2023-09-07 09:39:09,458 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ('3',)


True

In [45]:
class Address(base):
    __tablename__ = 'ADDRESS'
    pk = Column('PK', Integer, primary_key= True)
    name = Column('NAME', Text)
    fk = Column('FK', None, ForeignKey('USER.PK'))

    # representaion
    def __repr__(self):
        return f'PK: {self.pk}, NAME: {self.name}, FK: {self.fk}'

In [46]:
# 클래스 등록
base.metadata.create_all(engine)

2023-09-07 09:45:11,211 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:45:11,212 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-07 09:45:11,214 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:45:11,215 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ADDRESS")
2023-09-07 09:45:11,216 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:45:11,217 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ADDRESS")
2023-09-07 09:45:11,217 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:45:11,219 INFO sqlalchemy.engine.Engine 
CREATE TABLE "ADDRESS" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	"FK" INTEGER, 
	PRIMARY KEY ("PK"), 
	FOREIGN KEY("FK") REFERENCES "USER" ("PK")
)


2023-09-07 09:45:11,220 INFO sqlalchemy.engine.Engine [no key 0.00096s] ()
2023-09-07 09:45:11,221 INFO sqlalchemy.engine.Engine COMMIT


In [47]:
sess.add(Address(name = '주소1', fk = user1.pk))

In [48]:
sess.commit()

2023-09-07 09:45:57,957 INFO sqlalchemy.engine.Engine INSERT INTO "ADDRESS" ("NAME", "FK") VALUES (?, ?)
2023-09-07 09:45:57,958 INFO sqlalchemy.engine.Engine [generated in 0.00143s] ('주소1', 1)
2023-09-07 09:45:57,961 INFO sqlalchemy.engine.Engine COMMIT


In [51]:
address1 = sess.query(Address).one()

2023-09-07 09:47:20,118 INFO sqlalchemy.engine.Engine SELECT "ADDRESS"."PK" AS "ADDRESS_PK", "ADDRESS"."NAME" AS "ADDRESS_NAME", "ADDRESS"."FK" AS "ADDRESS_FK" 
FROM "ADDRESS"
2023-09-07 09:47:20,118 INFO sqlalchemy.engine.Engine [cached since 71.37s ago] ()


In [53]:
for user in userList:
    if user.pk == address1.fk:
        print(user, address1)

PK: 1, NAME: 다른값 PK: 1, NAME: 주소1, FK: 1


In [92]:
# 초기화
engine.dispose()
base.metadata.clear()
base.registry.dispose()

In [93]:
# CITY, SUPPLIER, PART, SELLS => ORM으로 만들기
base = declarative_base()

In [94]:
base.metadata.tables
engine = create_engine('sqlite:///:memory:', echo = True)

In [95]:
class CITY(base):    
    __tablename__ = 'CITY'
    cno = Column('CNO', Integer, primary_key= True)
    cname = Column('CNAME', Text)

    def __repr__(self):
        return f'CNO: {self.cno}, CNAME: {self.cname}'

class SUPPLIER(base):    
    __tablename__ = 'SUPPLIER'
    sno = Column('SNO', Integer, primary_key= True)
    sname = Column('SNAME', Text)
    cno = Column('CNO', Integer, nullable= False) # ForeignKey('CIIY.CNO')

    def __repr__(self):
        return f'SNO: {self.sno}, SNAME: {self.sname}, CNO: {self.cno}'
        
class PART(base):    
    __tablename__ = 'PART'
    pno = Column('PNO', Integer, primary_key= True)
    pname = Column('PNAME', Text)

    def __repr__(self):
        return f'PNO: {self.pno}, PNAME: {self.pname}'
    
class SELLS(base):    
    __tablename__ = 'SELLS'
    sno = Column('SNO', Integer, primary_key= True)  # ForeignKey('SUPPLIER.SNO')
    pno = Column('PNO', Integer, nullable=False) # ForeignKey('PART.PNO')
    price = Column('PRICE', Integer, nullable=False)

    def __repr__(self):
        return f'SNO: {self.sno}, PNO: {self.pno}, price: {self.price}'    
                

In [96]:
base.metadata.create_all(engine)

2023-09-07 10:16:55,339 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:16:55,340 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CITY")
2023-09-07 10:16:55,340 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:16:55,340 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CITY")
2023-09-07 10:16:55,341 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:16:55,342 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SUPPLIER")
2023-09-07 10:16:55,343 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:16:55,344 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SUPPLIER")
2023-09-07 10:16:55,345 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:16:55,346 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("PART")
2023-09-07 10:16:55,347 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:16:55,348 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("PART")
2023-09-07 10:16:55,349 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-0

In [97]:
Session = sessionmaker(engine)
sess = Session()

In [98]:
sess.add_all([CITY(cname = 'London'), CITY(cname = 'Paris'), CITY(cname = 'Rome'), CITY(cname = 'Vienna')])


In [99]:
sess.commit()

2023-09-07 10:16:59,290 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:16:59,291 INFO sqlalchemy.engine.Engine INSERT INTO "CITY" ("CNAME") VALUES (?) RETURNING "CNO"
2023-09-07 10:16:59,292 INFO sqlalchemy.engine.Engine [generated in 0.00008s (insertmanyvalues) 1/4 (ordered; batch not supported)] ('London',)
2023-09-07 10:16:59,293 INFO sqlalchemy.engine.Engine INSERT INTO "CITY" ("CNAME") VALUES (?) RETURNING "CNO"
2023-09-07 10:16:59,293 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/4 (ordered; batch not supported)] ('Paris',)
2023-09-07 10:16:59,294 INFO sqlalchemy.engine.Engine INSERT INTO "CITY" ("CNAME") VALUES (?) RETURNING "CNO"
2023-09-07 10:16:59,295 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/4 (ordered; batch not supported)] ('Rome',)
2023-09-07 10:16:59,296 INFO sqlalchemy.engine.Engine INSERT INTO "CITY" ("CNAME") VALUES (?) RETURNING "CNO"
2023-09-07 10:16:59,297 INFO sqlalchemy.engine.Engine [insertmanyvalues 4/4 (ordered; batch not supported)] 

In [100]:
sess.query(CITY).all()

2023-09-07 10:17:01,036 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:17:01,038 INFO sqlalchemy.engine.Engine SELECT "CITY"."CNO" AS "CITY_CNO", "CITY"."CNAME" AS "CITY_CNAME" 
FROM "CITY"
2023-09-07 10:17:01,039 INFO sqlalchemy.engine.Engine [generated in 0.00053s] ()


[CNO: 1, CNAME: London,
 CNO: 2, CNAME: Paris,
 CNO: 3, CNAME: Rome,
 CNO: 4, CNAME: Vienna]

In [101]:
sess.add_all([SUPPLIER(sname = 'Smith', cno = 1),\
              SUPPLIER(sname = 'Jones', cno = 2),\
              SUPPLIER(sname = 'Adams', cno = 1),\
              SUPPLIER(sname = 'Blake', cno = 3)])
sess.commit()
sess.query(SUPPLIER).all()

2023-09-07 10:17:02,463 INFO sqlalchemy.engine.Engine INSERT INTO "SUPPLIER" ("SNAME", "CNO") VALUES (?, ?) RETURNING "SNO"
2023-09-07 10:17:02,464 INFO sqlalchemy.engine.Engine [generated in 0.00009s (insertmanyvalues) 1/4 (ordered; batch not supported)] ('Smith', 1)
2023-09-07 10:17:02,465 INFO sqlalchemy.engine.Engine INSERT INTO "SUPPLIER" ("SNAME", "CNO") VALUES (?, ?) RETURNING "SNO"
2023-09-07 10:17:02,466 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/4 (ordered; batch not supported)] ('Jones', 2)
2023-09-07 10:17:02,466 INFO sqlalchemy.engine.Engine INSERT INTO "SUPPLIER" ("SNAME", "CNO") VALUES (?, ?) RETURNING "SNO"
2023-09-07 10:17:02,467 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/4 (ordered; batch not supported)] ('Adams', 1)
2023-09-07 10:17:02,467 INFO sqlalchemy.engine.Engine INSERT INTO "SUPPLIER" ("SNAME", "CNO") VALUES (?, ?) RETURNING "SNO"
2023-09-07 10:17:02,468 INFO sqlalchemy.engine.Engine [insertmanyvalues 4/4 (ordered; batch not supported)] ('Blake',

[SNO: 1, SNAME: Smith, CNO: 1,
 SNO: 2, SNAME: Jones, CNO: 2,
 SNO: 3, SNAME: Adams, CNO: 1,
 SNO: 4, SNAME: Blake, CNO: 3]

In [102]:
sess.add_all([PART(pname = 'Screw'),\
              PART(pname = 'Nut'),\
              PART(pname = 'Bolt'),\
              PART(pname = 'Cam')])
sess.commit()
sess.query(PART).all()

2023-09-07 10:17:03,848 INFO sqlalchemy.engine.Engine INSERT INTO "PART" ("PNAME") VALUES (?) RETURNING "PNO"
2023-09-07 10:17:03,849 INFO sqlalchemy.engine.Engine [generated in 0.00009s (insertmanyvalues) 1/4 (ordered; batch not supported)] ('Screw',)
2023-09-07 10:17:03,850 INFO sqlalchemy.engine.Engine INSERT INTO "PART" ("PNAME") VALUES (?) RETURNING "PNO"
2023-09-07 10:17:03,851 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/4 (ordered; batch not supported)] ('Nut',)
2023-09-07 10:17:03,852 INFO sqlalchemy.engine.Engine INSERT INTO "PART" ("PNAME") VALUES (?) RETURNING "PNO"
2023-09-07 10:17:03,852 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/4 (ordered; batch not supported)] ('Bolt',)
2023-09-07 10:17:03,854 INFO sqlalchemy.engine.Engine INSERT INTO "PART" ("PNAME") VALUES (?) RETURNING "PNO"
2023-09-07 10:17:03,854 INFO sqlalchemy.engine.Engine [insertmanyvalues 4/4 (ordered; batch not supported)] ('Cam',)
2023-09-07 10:17:03,856 INFO sqlalchemy.engine.Engine COMMIT
2023

[PNO: 1, PNAME: Screw,
 PNO: 2, PNAME: Nut,
 PNO: 3, PNAME: Bolt,
 PNO: 4, PNAME: Cam]

In [103]:
sess.add_all([SELLS(pno = 1, price = 10),\
              SELLS(pno = 2, price = 8),\
              SELLS(pno = 4, price = 38),\
              SELLS(pno = 1, price = 11),\
              SELLS(pno = 3, price = 6),\
              SELLS(pno = 2, price = 7),\
              SELLS(pno = 3, price = 4),\
              SELLS(pno = 4, price = 45)])
sess.commit()
sess.query(SELLS).all()

2023-09-07 10:17:05,501 INFO sqlalchemy.engine.Engine INSERT INTO "SELLS" ("PNO", "PRICE") VALUES (?, ?) RETURNING "SNO"
2023-09-07 10:17:05,502 INFO sqlalchemy.engine.Engine [generated in 0.00009s (insertmanyvalues) 1/8 (ordered; batch not supported)] (1, 10)
2023-09-07 10:17:05,503 INFO sqlalchemy.engine.Engine INSERT INTO "SELLS" ("PNO", "PRICE") VALUES (?, ?) RETURNING "SNO"
2023-09-07 10:17:05,504 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/8 (ordered; batch not supported)] (2, 8)
2023-09-07 10:17:05,505 INFO sqlalchemy.engine.Engine INSERT INTO "SELLS" ("PNO", "PRICE") VALUES (?, ?) RETURNING "SNO"
2023-09-07 10:17:05,505 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/8 (ordered; batch not supported)] (4, 38)
2023-09-07 10:17:05,507 INFO sqlalchemy.engine.Engine INSERT INTO "SELLS" ("PNO", "PRICE") VALUES (?, ?) RETURNING "SNO"
2023-09-07 10:17:05,508 INFO sqlalchemy.engine.Engine [insertmanyvalues 4/8 (ordered; batch not supported)] (1, 11)
2023-09-07 10:17:05,509 INFO 

[SNO: 1, PNO: 1, price: 10,
 SNO: 2, PNO: 2, price: 8,
 SNO: 3, PNO: 4, price: 38,
 SNO: 4, PNO: 1, price: 11,
 SNO: 5, PNO: 3, price: 6,
 SNO: 6, PNO: 2, price: 7,
 SNO: 7, PNO: 3, price: 4,
 SNO: 8, PNO: 4, price: 45]

In [109]:
cno = sess.query(CITY).where(CITY.cno == 1).one().cno

2023-09-07 10:25:14,061 INFO sqlalchemy.engine.Engine SELECT "CITY"."CNO" AS "CITY_CNO", "CITY"."CNAME" AS "CITY_CNAME" 
FROM "CITY" 
WHERE "CITY"."CNO" = ?
2023-09-07 10:25:14,061 INFO sqlalchemy.engine.Engine [cached since 16.38s ago] (1,)


In [105]:
sess.query(SUPPLIER).filter(SUPPLIER.sname == 'Smith').one()

2023-09-07 10:20:34,258 INFO sqlalchemy.engine.Engine SELECT "SUPPLIER"."SNO" AS "SUPPLIER_SNO", "SUPPLIER"."SNAME" AS "SUPPLIER_SNAME", "SUPPLIER"."CNO" AS "SUPPLIER_CNO" 
FROM "SUPPLIER" 
WHERE "SUPPLIER"."SNAME" = ?
2023-09-07 10:20:34,258 INFO sqlalchemy.engine.Engine [generated in 0.00101s] ('Smith',)


SNO: 1, SNAME: Smith, CNO: 1

In [ ]:
base.metadata.create_all(engine)

In [127]:
from sqlalchemy.orm import relationship

engine.dispose()
base.metadata.clear()
base.registry.dispose()

In [130]:
class CITY(base):    
    __tablename__ = 'CITY'
    __table_args__ = {'extend_existing': True}
    cno = Column('CNO', Integer, primary_key= True)
    cname = Column('CNAME', Text)
    #nodes = relationship('SUPPLIER', back_populates='CITY', uselist = True)

    def __repr__(self):
        return f'CNO: {self.cno}, CNAME: {self.cname}'

class SUPPLIER(base):    
    __tablename__ = 'SUPPLIER'
    __table_args__ = {'extend_existing': True}
    sno = Column('SNO', Integer, primary_key= True)
    sname = Column('SNAME', Text)
    cno = Column('CNO', Integer, ForeignKey('CITY.CNO'))

    def __repr__(self):
        return f'SNO: {self.sno}, SNAME: {self.sname}, CNO: {self.cno}'
        
class PART(base):    
    __tablename__ = 'PART'
    __table_args__ = {'extend_existing': True}
    pno = Column('PNO', Integer, primary_key= True)
    pname = Column('PNAME', Text)

    def __repr__(self):
        return f'PNO: {self.pno}, PNAME: {self.pname}'
    
class SELLS(base):    
    __tablename__ = 'SELLS'
    __table_args__ = {'extend_existing': True}
    pk = Column('PK', Integer, primary_key = True)
    sno = Column('SNO', Integer, ForeignKey('SUPPLIER.SNO'))
    pno = Column('PNO', Integer, ForeignKey('PART.PNO'))
    price = Column('PRICE', Integer, nullable=False)

    def __repr__(self):
        return f'PK: {self.pk}, SNO: {self.sno}, PNO: {self.pno}, price: {self.price}'    
                

C:\Users\GyeongJun\AppData\Local\Temp\ipykernel_1192\2633134379.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.CITY, and will be replaced in the string-lookup table.
  class CITY(base):
C:\Users\GyeongJun\AppData\Local\Temp\ipykernel_1192\2633134379.py:11: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.SUPPLIER, and will be replaced in the string-lookup table.
  class SUPPLIER(base):
C:\Users\GyeongJun\AppData\Local\Temp\ipykernel_1192\2633134379.py:21: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.PART, and will be replaced in the string-lookup table.
  class PART(base):
C:\Users\GyeongJun\AppData\Local\Temp\ipykernel_1192\2633134379.py:30: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.SELLS, and will be replaced in the strin

In [131]:
base.metadata.create_all(engine)

2023-09-07 10:44:14,341 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:44:14,342 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CITY")
2023-09-07 10:44:14,343 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:44:14,345 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CITY")
2023-09-07 10:44:14,346 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:44:14,347 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SUPPLIER")
2023-09-07 10:44:14,347 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:44:14,349 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SUPPLIER")
2023-09-07 10:44:14,349 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:44:14,350 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("PART")
2023-09-07 10:44:14,350 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:44:14,350 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("PART")
2023-09-07 10:44:14,351 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-0

In [132]:
base.metadata.tables

FacadeDict({'CITY': Table('CITY', MetaData(), Column('CNO', Integer(), table=<CITY>, primary_key=True, nullable=False), Column('CNAME', Text(), table=<CITY>), schema=None), 'SUPPLIER': Table('SUPPLIER', MetaData(), Column('SNO', Integer(), table=<SUPPLIER>, primary_key=True, nullable=False), Column('SNAME', Text(), table=<SUPPLIER>), Column('CNO', Integer(), ForeignKey('CITY.CNO'), table=<SUPPLIER>), schema=None), 'PART': Table('PART', MetaData(), Column('PNO', Integer(), table=<PART>, primary_key=True, nullable=False), Column('PNAME', Text(), table=<PART>), schema=None), 'SELLS': Table('SELLS', MetaData(), Column('PK', Integer(), table=<SELLS>, primary_key=True, nullable=False), Column('SNO', Integer(), ForeignKey('SUPPLIER.SNO'), table=<SELLS>), Column('PNO', Integer(), ForeignKey('PART.PNO'), table=<SELLS>), Column('PRICE', Integer(), table=<SELLS>, nullable=False), schema=None)})

In [193]:
sess.rollback()
sess.close()

In [194]:
engine.dispose()
base.metadata.clear()
base.registry.dispose()

In [192]:
engine = create_engine('sqlite:///:memory:', echo = True)
base = declarative_base()
Session = sessionmaker(engine)
sess= Session()

In [ ]:
# 기존 DB에서 불러오기
base.metadata.reflect(engine)
base.metadata.tables['테이블이름']

class 어쩌구():
    __table__= base.metadata.tables['테이블이름']

# relationship을 만드려면 DB에 PK/FK관계까 있어야함.

In [189]:
class Artist(base):
    __tablename__ = 'ARTIST'
    pk = Column('PK', Integer, primary_key= True)
    name = Column('NAME', Text)
    # relationship 관계를 위해 쌍을 설정
    albums = relationship('Album', back_populates= 'artist', uselist=True)
    
    def addAlbum(self, name):
        album = Album(name = name)
        sess.add(album)
        sess.commit()
        self.albums.append(album)

    def removeAlbum(self, name):
        album = sess.query(Album).filter(Album.name == name).one()
        self.albums.index(album)
        
        sess.commit()

    def __repr__(self):
        return f'{self.name}, {len(self.albums)}'
    
class Album(base):
    __tablename__ = 'ALBUM'
    pk = Column('PK', Integer, primary_key= True)
    name = Column('NAME', Text)    
    fk = Column('FK', Integer, ForeignKey('ARTIST.PK'))
    artist = relationship('Artist', back_populates='albums')

    def __repr__(self):
        return f'{self.name}, {len(self.artist.albums)}'

Exception ignored in: <function _ConnectionRecord.checkout.<locals>.<lambda> at 0x000001FB20A61F70>
Traceback (most recent call last):
  File "c:\Users\GyeongJun\anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 732, in <lambda>
  File "c:\Users\GyeongJun\anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 1043, in _finalize_fairy
AttributeError: 'NoneType' object has no attribute 'dbapi_connection'


SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (2268504082.py, line 15)

In [ ]:
base.metadata.create_all(engine)

In [182]:
artist1 = Artist(name = '가수1')
sess.add(artist1)
sess.commit()

2023-09-07 11:21:41,163 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:21:41,164 INFO sqlalchemy.engine.Engine INSERT INTO "ARTIST" ("NAME") VALUES (?)
2023-09-07 11:21:41,165 INFO sqlalchemy.engine.Engine [generated in 0.00084s] ('가수1',)
2023-09-07 11:21:41,166 INFO sqlalchemy.engine.Engine COMMIT


In [183]:
artist1.addAlbum('앨범1')

2023-09-07 11:21:42,219 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:21:42,220 INFO sqlalchemy.engine.Engine INSERT INTO "ALBUM" ("NAME", "FK") VALUES (?, ?)
2023-09-07 11:21:42,220 INFO sqlalchemy.engine.Engine [generated in 0.00056s] ('앨범1', None)
2023-09-07 11:21:42,222 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 11:21:42,223 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:21:42,224 INFO sqlalchemy.engine.Engine SELECT "ARTIST"."PK" AS "ARTIST_PK", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."PK" = ?
2023-09-07 11:21:42,225 INFO sqlalchemy.engine.Engine [generated in 0.00073s] (1,)
2023-09-07 11:21:42,227 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE ? = "ALBUM"."FK"
2023-09-07 11:21:42,228 INFO sqlalchemy.engine.Engine [generated in 0.00114s] (1,)


In [184]:
artist1.albums[0]

2023-09-07 11:21:43,732 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME" 
FROM "ALBUM" 
WHERE "ALBUM"."PK" = ?
2023-09-07 11:21:43,733 INFO sqlalchemy.engine.Engine [generated in 0.00095s] (1,)
2023-09-07 11:21:43,736 INFO sqlalchemy.engine.Engine UPDATE "ALBUM" SET "FK"=? WHERE "ALBUM"."PK" = ?
2023-09-07 11:21:43,737 INFO sqlalchemy.engine.Engine [generated in 0.00100s] (1, 1)
2023-09-07 11:21:43,740 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE "ALBUM"."PK" = ?
2023-09-07 11:21:43,741 INFO sqlalchemy.engine.Engine [generated in 0.00103s] (1,)


앨범1, 1

In [ ]:
# 앨범을 삭제하는 코드를 만들기 (11시 20분)
artist1.removeAlbum('앨범1')

In [212]:
sess.rollback()
sess.close()
engine.dispose()
base.metadata.clear()
base.registry.dispose()

In [213]:
engine = create_engine('sqlite:///:memory:', echo = True)
base = declarative_base()
Session = sessionmaker(engine)
sess= Session()

In [215]:
# Artist 1:N Ablums (관계)
# Post N:M HashTag => 관계를 정의하고 있는 별도의 테이블을 써야함.
# Post               Tags              Hashtag
# tags-------------post/hashtag----------posts
# Posts.tags[0].hashtag -------------> Hashtag

class Post(base):
    __tablename__ = 'POST'
    __table_args__ = {'extend_existing': True}
    pk = Column('PK', Integer, primary_key= True)
    content = Column('CONTENT', Text)
    # tags는 Tags라는 Table과 관계를 갖고 있고, post를 통해 연결됨
    tags = relationship('Tags', back_populates='post', uselist=True)

    def addTags(self, tags):
        tagList = []
        for tag in tags:
            rst = sess.query(Hashtag).filter(Hashtag.name == tag)
            if rst.count() > 0:
                tagList.append(rst.one())
            else:
                newTag = Hashtag(name = tag)
                sess.add(newTag)
                sess.commit()
                tagList.append(newTag)
        for tag in tagList:
            sess.add(Tags(fk1 = self.pk, fk2 = tag.pk))
            sess.commit()
            tag.cnt += 1

    def __repr__(self):
        return f'{self.pk}, {len(self.tags)}'

class Hashtag(base):
    __tablename__ = 'HASHTAG'
    __table_args__ = {'extend_existing': True}
    pk = Column('PK', Integer, primary_key= True)
    name = Column('NAME', Text)
    cnt = Column('CNT', Integer, server_default='0')
    posts = relationship('Tags', back_populates='hashtag', uselist=True)

    def __repr__(self):
        return f'{self.pk}, {self.name}, {len(self.posts)}'    

class Tags(base):
    __tablename__ = 'TAGS'
    __table_args__ = {'extend_existing': True}
    pk = Column('PK', Integer, primary_key= True)
    fk1 = Column('FK1', Integer, None, ForeignKey('POST.PK'))
    fk2 = Column('FK2', Integer, None, ForeignKey('HASHTAG.PK'))
    post = relationship('Post', back_populates='tags')
    hashtag = relationship('Hashtag', back_populates='posts')

C:\Users\GyeongJun\AppData\Local\Temp\ipykernel_1192\1890848086.py:7: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Post, and will be replaced in the string-lookup table.
  class Post(base):


In [216]:
base.metadata.create_all(engine)

2023-09-07 11:53:11,868 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:53:11,869 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("POST")
2023-09-07 11:53:11,870 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:53:11,871 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("POST")
2023-09-07 11:53:11,873 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:53:11,875 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("HASHTAG")
2023-09-07 11:53:11,876 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:53:11,877 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("HASHTAG")
2023-09-07 11:53:11,877 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:53:11,878 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("TAGS")
2023-09-07 11:53:11,878 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:53:11,879 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("TAGS")
2023-09-07 11:53:11,879 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 

In [199]:
post1 = Post(content = '내용1')
sess.add(post1)

In [200]:
sess.commit()

2023-09-07 11:39:22,721 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:39:22,722 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?)
2023-09-07 11:39:22,724 INFO sqlalchemy.engine.Engine [generated in 0.00062s] ('내용1',)
2023-09-07 11:39:22,725 INFO sqlalchemy.engine.Engine COMMIT


In [201]:
tag1 = Hashtag(name = '해시태그1')
sess.add(tag1)
sess.commit()

2023-09-07 11:40:07,600 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:40:07,602 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME") VALUES (?)
2023-09-07 11:40:07,603 INFO sqlalchemy.engine.Engine [generated in 0.00141s] ('해시태그1',)
2023-09-07 11:40:07,607 INFO sqlalchemy.engine.Engine COMMIT


In [202]:
sess.add(Tags(fk1 = post1.pk, fk2 =tag1.pk))
sess.commit()

2023-09-07 11:40:43,836 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:40:43,838 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2023-09-07 11:40:43,838 INFO sqlalchemy.engine.Engine [generated in 0.00087s] (1,)
2023-09-07 11:40:43,841 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:40:43,842 INFO sqlalchemy.engine.Engine [generated in 0.00090s] (1,)
2023-09-07 11:40:43,845 INFO sqlalchemy.engine.Engine INSERT INTO "TAGS" ("FK1", "FK2") VALUES (?, ?)
2023-09-07 11:40:43,845 INFO sqlalchemy.engine.Engine [generated in 0.00096s] (1, 1)
2023-09-07 11:40:43,847 INFO sqlalchemy.engine.Engine COMMIT


In [204]:
len(post1.tags), post1.tags[0].hashtag.name

2023-09-07 11:41:09,608 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:41:09,609 INFO sqlalchemy.engine.Engine [cached since 25.77s ago] (1,)


(1, '해시태그1')

In [206]:
len(tag1.posts), tag1.posts[0].post.content

(1, '내용1')

In [217]:
sess.rollback()
sess.close()
engine.dispose()
base.metadata.clear()
base.registry.dispose()

engine = create_engine('sqlite:///:memory:', echo = True)
base = declarative_base()
Session = sessionmaker(engine)
sess= Session()

In [218]:
# Artist 1:N Ablums (관계)
# Post N:M HashTag => 관계를 정의하고 있는 별도의 테이블을 써야함.
# Post               Tags              Hashtag
# tags-------------post/hashtag----------posts
# Posts.tags[0].hashtag -------------> Hashtag

class Post(base):
    __tablename__ = 'POST'
    __table_args__ = {'extend_existing': True}
    pk = Column('PK', Integer, primary_key= True)
    content = Column('CONTENT', Text)
    # tags는 Tags라는 Table과 관계를 갖고 있고, post를 통해 연결됨
    tags = relationship('Tags', back_populates='post', uselist=True)

    # tag 추가
    def addTags(self, tags):
        tagList = []
        for tag in tags:
            rst = sess.query(Hashtag).filter(Hashtag.name == tag)
            if rst.count() > 0:
                tagList.append(rst.one())
            else:
                newTag = Hashtag(name = tag)
                sess.add(newTag)
                sess.commit()
                tagList.append(newTag)
        for tag in tagList:
            sess.add(Tags(fk1 = self.pk, fk2 = tag.pk))
            sess.commit()
            tag.cnt += 1

    # tag 삭제    
    def deleteTags(self, tags):
        del_tagList = []
        for tag in tags:
            rst = sess.query(Hashtag).filter(Hashtag.name == tag)
            if rst.count() == 0:
                continue
            else:
                newTag = sess.query(Hashtag).filter(Hashtag.name == tag).one()
                index = self.tags.index(newTag)
                

                sess.commit()
                del_tagList.append(newTag)

        for tag in del_tagList:
            sess.commit()
            tag.cnt -= 1

    def __repr__(self):
        return f'{self.pk}, {len(self.tags)}'
    

class Hashtag(base):
    __tablename__ = 'HASHTAG'
    __table_args__ = {'extend_existing': True}
    pk = Column('PK', Integer, primary_key= True)
    name = Column('NAME', Text)
    cnt = Column('CNT', Integer, server_default='0')
    posts = relationship('Tags', back_populates='hashtag', uselist=True)

    def __repr__(self):
        return f'{self.pk}, {self.name}, {len(self.posts)}'    
    
    # +1, -1 함수

class Tags(base):
    __tablename__ = 'TAGS'
    __table_args__ = {'extend_existing': True}
    pk = Column('PK', Integer, primary_key= True)
    fk1 = Column('FK1', Integer, None, ForeignKey('POST.PK'))
    fk2 = Column('FK2', Integer, None, ForeignKey('HASHTAG.PK'))
    post = relationship('Post', back_populates='tags')
    hashtag = relationship('Hashtag', back_populates='posts')

In [219]:
base.metadata.create_all(engine)

2023-09-07 11:54:27,783 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:54:27,784 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("POST")
2023-09-07 11:54:27,785 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:54:27,788 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("POST")
2023-09-07 11:54:27,789 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:54:27,792 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("HASHTAG")
2023-09-07 11:54:27,793 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:54:27,794 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("HASHTAG")
2023-09-07 11:54:27,794 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:54:27,795 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("TAGS")
2023-09-07 11:54:27,796 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:54:27,796 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("TAGS")
2023-09-07 11:54:27,797 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 

In [220]:
post2 = Post(content = '내용2')
sess.add(post2)
sess.commit()
post2.addTags(['해시태그2', '해시태그3'])

2023-09-07 11:55:46,594 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:55:46,596 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?)
2023-09-07 11:55:46,597 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ('내용2',)
2023-09-07 11:55:46,599 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 11:55:46,600 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:55:46,605 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?) AS anon_1
2023-09-07 11:55:46,606 INFO sqlalchemy.engine.Engine [generated in 0.00114s] ('해시태그2',)
2023-09-07 11:55:46,608 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME") VALUES (?) RETURNING "PK", "CNT"
2023-09-07 11:55:46,609 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ('해시태그2',)
2023-09-07 11:55:46,611 INFO sqlalchemy.engine.Engine COMMIT
2023-0

In [221]:
post3 = Post(content = '내용3')
sess.add(post3)
sess.commit()
post3.addTags(['해시태그1', '해시태그2'])

2023-09-07 11:56:13,117 INFO sqlalchemy.engine.Engine UPDATE "HASHTAG" SET "CNT"=? WHERE "HASHTAG"."PK" = ?
2023-09-07 11:56:13,118 INFO sqlalchemy.engine.Engine [cached since 26.49s ago] (1, 2)
2023-09-07 11:56:13,120 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?)
2023-09-07 11:56:13,121 INFO sqlalchemy.engine.Engine [cached since 26.52s ago] ('내용3',)
2023-09-07 11:56:13,123 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 11:56:13,123 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:56:13,125 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?) AS anon_1
2023-09-07 11:56:13,126 INFO sqlalchemy.engine.Engine [cached since 26.52s ago] ('해시태그1',)
2023-09-07 11:56:13,128 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME") VALUES (?) RETURNING "PK", "CNT"
2023-09-07 11:56:13,128 INFO

In [222]:
tag1 = sess.query(Hashtag).where(Hashtag.name == '해시태그2').one()

2023-09-07 11:56:43,520 INFO sqlalchemy.engine.Engine UPDATE "HASHTAG" SET "CNT"=? WHERE "HASHTAG"."PK" = ?
2023-09-07 11:56:43,521 INFO sqlalchemy.engine.Engine [cached since 56.89s ago] (2, 1)
2023-09-07 11:56:43,522 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?
2023-09-07 11:56:43,523 INFO sqlalchemy.engine.Engine [cached since 30.39s ago] ('해시태그2',)


In [223]:
for post in tag1.posts:
    print(post.post.content)

2023-09-07 11:57:08,262 INFO sqlalchemy.engine.Engine SELECT "TAGS"."PK" AS "TAGS_PK", "TAGS"."FK1" AS "TAGS_FK1", "TAGS"."FK2" AS "TAGS_FK2" 
FROM "TAGS" 
WHERE ? = "TAGS"."FK2"
2023-09-07 11:57:08,265 INFO sqlalchemy.engine.Engine [generated in 0.00205s] (1,)
2023-09-07 11:57:08,266 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2023-09-07 11:57:08,267 INFO sqlalchemy.engine.Engine [cached since 81.65s ago] (1,)
내용2
2023-09-07 11:57:08,268 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2023-09-07 11:57:08,269 INFO sqlalchemy.engine.Engine [cached since 81.65s ago] (2,)
내용3


In [240]:
sess.query(Hashtag).filter(Hashtag.name == '해시태그1').all()

2023-09-07 12:26:31,229 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?
2023-09-07 12:26:31,231 INFO sqlalchemy.engine.Engine [cached since 1818s ago] ('해시태그1',)


[]

In [239]:
sess.query(Hashtag).filter(Hashtag.name == '해시태그1').delete()

2023-09-07 12:26:19,777 INFO sqlalchemy.engine.Engine DELETE FROM "HASHTAG" WHERE "HASHTAG"."NAME" = ?
2023-09-07 12:26:19,777 INFO sqlalchemy.engine.Engine [generated in 0.00151s] ('해시태그1',)


1